In [104]:
import pandas as pd
import numpy as np
import json
import sqlite3

- Cargue en DataFrames las cuatro tablas (3 csv, 1 json) que contiene la informacion originaria.

In [359]:
df_amazon_original = pd.read_csv(r'C:\Users\ornal\Desktop\CursoDataScience\PI01_DATA05\Datasets\amazon_prime_titles.csv')
df_disney_original = pd.read_csv(r'C:\Users\ornal\Desktop\CursoDataScience\PI01_DATA05\Datasets\disney_plus_titles.csv')
df_hulu_original = pd.read_csv(r'C:\Users\ornal\Desktop\CursoDataScience\PI01_DATA05\Datasets\hulu_titles.csv')
df_netflix_original = pd.read_json(r'C:\Users\ornal\Desktop\CursoDataScience\PI01_DATA05\Datasets\netflix_titles.json')

In [842]:
df_amazon_original['plataforma'] = 1
df_disney_original['plataforma'] = 2
df_hulu_original['plataforma'] = 3
df_netflix_original['plataforma'] = 4

- Junto los origenes en un solo DataFrame

In [862]:
df_original = pd.concat([df_amazon_original,df_disney_original,df_hulu_original,df_netflix_original])

In [887]:
df_original1 = df_original[df_original['cast'].isna()]

In [888]:
df_original2 = df_original.dropna(subset='cast')

Cambiando datos incorrectos

In [ ]:
df_original['cast'] = df_original['cast'].replace('1', 'N/A')
df_original['cast'] = df_original['cast'].replace("'Najite Dede, Jude Chukwuka, Taiwo Arimoro, Odenike Odetola, Funmi Eko, Keppy Ekpenyong", "Najite Dede, Jude Chukwuka, Taiwo Arimoro, Odenike Odetola, Funmi Eko, Keppy Ekpenyong")
df_original['cast'] = df_original['cast'].replace('1, 2, 3', 'N/A')
df_original['cast'] = df_original['cast'].replace('10 singers from Real Girls Project, Sung Hoon, Park Chulmin', 'Sung Hoon, Park Chulmin')
df_original['cast'] = df_original['cast'].replace('50 Cent, Ryan Phillippe, Bruce Willis, Rory Markham, Jenna Dewan, Brett Granstaff, Randy Couture, Susie Abromeit, Ron Turner, James Remar', 'Ryan Phillippe, Bruce Willis, Rory Markham, Jenna Dewan, Brett Granstaff, Randy Couture, Susie Abromeit, Ron Turner, James Remar, 50 Cent')

In [ ]:
# df["id_valor1"] = df["valor1"].apply(lambda x: df_b[df_b["id"] == x]["val"].item() if df_b["id"].isin([x]).any() else "NaN")
df_original1['title'].apply(lambda x: df_original2[df_original2['title']==x]['cast'])

- Salvo todos los DataFrames en una BD SQLite 'origen_audiovisual.db'
Usando SQLiteStudio
Es mas comodo revisar visualmente las tablas usando los grids 

In [894]:
conn = sqlite3.connect('origen_audiovisual.db')
df_amazon_original.to_sql('amazon', conn, if_exists='replace', index=True)
df_disney_original.to_sql('disney', conn, if_exists='replace', index=True)
df_hulu_original.to_sql('hulu', conn, if_exists='replace', index=True)
df_netflix_original.to_sql('netflix', conn, if_exists='replace', index=True)
df_original.to_sql('todo', conn, if_exists='replace', index=True)
conn.close()

Aqui revise las info de las tablas para ver los nulos donde estan

In [ ]:
#df_amazon_original.info()
#df_disney_original.info()
#df_hulu_original.info()
#df_netflix_original.info()

Traigo de vuelta las tablas originales desde SQLite, con algunos campos cast rellenados

In [ ]:
conn = sqlite3.connect('origen_audiovisual.db')
df_original.read_sql('todo', conn, if_exists='replace', index=True)
conn.close()

Hago una copia de los DataFrames originales y ya desecho los campos que asumo no me hacen falta de acuerdo a lo que me estan pidiendo

In [620]:
df_amazon = df_amazon_original[['title','cast','type','release_year','duration','listed_in']]

In [641]:
df_hulu = df_hulu_original[['title','cast','type','release_year','duration','listed_in']]

In [622]:
df_disney = df_disney_original[['title','cast','type','release_year','duration','listed_in']]

In [623]:
df_netflix = df_netflix_original[['title','cast','type','release_year','duration','listed_in']]

Primero tengo que separar el valor entero de la descripcion en la columna duration y dejar solo el entero

In [624]:
def separa(x):
  x = str(x)
  if x != None:
    return x.split(' ')[0]
  else:
    return x 

Antes de correr la funcion para separar el entero del texto relleno los nulos de esa fila en la unica tabla que tenia nulos ahi
Deje para mas adelante a resolucion de los 0 que acabo de agregar 

In [ ]:
df_hulu.duration = df_hulu['duration'].fillna(0)

Ejecuto la funcion separa en todos los DataFrames y ya tengo la columna duration con numeros

In [ ]:
df_amazon.duration = df_amazon.duration.apply(separa)
df_disney.duration = df_disney.duration.apply(separa)
df_hulu.duration = df_hulu.duration.apply(separa)
df_netflix.duration = df_netflix.duration.apply(separa)

Despues de analizar los nulos en cast, existe como sacar algunos porque existen titles repetidos que en un caso tiene cast y en otro no. De momento todos los nulos se va con la etiqueta N/A

In [ ]:
df_amazon.cast = df_amazon['cast'].fillna('N/A')
df_hulu.cast = df_hulu['cast'].fillna('N/A')
df_disney.cast = df_disney['cast'].fillna('N/A')
df_netflix.cast = df_netflix['cast'].fillna('N/A')

Creo el DataFrame df_plataforma (Clasificador)

In [649]:
df_plataforma = pd.DataFrame(['Amazon','Disney','Hulu','Netflix'],index=range(1,5),columns=['plataforma'])

Agrego una columna a cada tabla y le pongo el valor que le corresponde en la tabla df_plataforma

In [ ]:
df_amazon['plataforma'] = 1
df_disney['plataforma'] = 2
df_hulu['plataforma'] = 3
df_netflix['plataforma'] = 4

Creo un dataframe unico concatenando los 4 dataframes que me entran. Este va a dar origen a la tabla principal que se va a llamar 'audiovisual'

In [897]:
df = pd.concat([df_amazon,df_disney,df_hulu,df_netflix])

Creo una columna con el id empezando por 1

In [898]:
df['id'] = range(1,df.shape[0]+1)

pongo la columna id de df como indice

In [899]:
df = df.set_index('id')

Preparo los Clasificadores de 'genero' y 'artista'

Hago arreglos en el campo 'listed_in'

In [687]:
for i in range(1,df.shape[0]+1): #recorro df
  lista_genero = df.at[i,'listed_in'].split(',') #pongo en una lista todos los generos que esten en lised_in
  nueva_lista = []                               # creo una lista vacia
  for genero in lista_genero:                      # recorro cada elemento de la lista donde estan los generos
    genero = genero.strip()
    match genero:                                 # entro en un case para arreglar
      case 'Action & Adventure':                  # aqui separo Action
        nueva_lista.append('Action')
        nueva_lista.append('Adventure')
      case 'Action-Adventure':
        nueva_lista.append('Action')
        nueva_lista.append('Adventure') 
      case 'Animals & Nature':   
        nueva_lista.append('Animals')
        nueva_lista.append('nature') 
      case 'Anime':
        nueva_lista.append('Animation')
      case 'Anime Features':
        nueva_lista.append('Animation')
      case 'Anime Series':
        nueva_lista.append('Animation')
      case 'Classic & Cult TV':
        nueva_lista.append('Classic')    
      case 'Classic Movies':  
        nueva_lista.append('Classic')
      case 'Classics':  
        nueva_lista.append('Classic')
      case 'Comedies':
        nueva_lista.append('Comedy')  
      case 'Crime TV Shows':
        nueva_lista.append('Crime')  
      case 'Documentaries':
        nueva_lista.append('Documentary')
      case 'Docuseries':
        nueva_lista.append('Documentary')
      case 'Dramas':
        nueva_lista.append('Drama')          
      case 'Faith & Spirituality':
        nueva_lista.append('Faith and Spirituality') 
      case 'Game Show / Competition':
        nueva_lista.append('Game Shows') 
      case 'Historical':
        nueva_lista.append('History')  
      case 'Horror Movies':
        nueva_lista.append('Horror')
      case 'International Movies':
        nueva_lista.append('International')                    
      case 'International TV Shows':
        nueva_lista.append('International') 
      case "Kids' TV":
        nueva_lista.append('Kids')  
      case 'LGBTQ Movies':
        nueva_lista.append('LGBTQ')                          
      case 'LGBTQ+':
        nueva_lista.append('LGBTQ')  
      case _:
        nueva_lista.append(genero)
  df.at[i,'listed_in'] = ','.join(nueva_lista)

A partir del campo 'listed_in' voy a obtener una lista llamada 'generos' que me va a servir para crear el clasificador de generos

In [688]:
listGenero = []
listGenero = list(df['listed_in'])
generos = []

for elem in listGenero:
  elem = str(elem)
  gen = elem.split(',')
  for subelem in gen:
    subelem = subelem.strip('"')
    subelem = subelem.strip(" #")
    
    if subelem not in generos:
      generos.append(subelem)

Ya tengo en la lista genero, ahora creo el DataFrame df_genero

In [689]:
df_genero = pd.DataFrame(generos, index=range(1,len(generos)+1),columns=['nombre_genero'])

Le creo un id

In [690]:
df_genero['id'] = range(1,len(generos)+1)

lo pongo de primero

In [691]:
df_genero = df_genero[['id','nombre_genero']]

A partir del campo 'cast' voy a obtener una lista llamada 'artistas' que me va a servir para crear el clasificador de artistas

In [ ]:
df.info()

In [901]:
listArtista = []
listArtista = list(df['cast'])
artistas = []

for elem in listArtista:
  elem = str(elem)
  gen = elem.split(',')
  for subelem in gen:    
    if subelem not in artistas:
      artistas.append(subelem)

In [902]:
df_artista = pd.DataFrame(artistas, index=range(1,len(artistas)+1),columns=['nombre_artista'])

In [903]:
df_artista['id'] = range(1,df_artista.shape[0]+1)

In [904]:
df_artista = df_artista.set_index('id')

creo un dataframe para la relacion audiovisual - artista

In [905]:
df['id'] = range(1,df.shape[0]+1)

In [906]:
df_temp = df[['id','cast']]

Creo una lista de diccionarios que contiene la relacion de cada audiovisual con con cada uno de los artistas contenidos en la columna cast. Esta relacion es de muchos a muchos. Para ello me auxilio del clasificador df_artista
De aqui salgo con lista que es una lista de tuplas (nombre artista, id del audiovisual)

In [ ]:
lista = []
for i in range(1,df.shape[0]+1):
  lista_cast = df.at[i,'cast'].split(',')
  nueva_lista = []                               
  for cast in lista_cast:                      
    cast = cast.strip()
    lista.append((cast,i))

In [907]:
df_artista['id'] = range(1,df_artista.shape[0]+1)

In [908]:
df_artista = df_artista[['id','nombre_artista']]

Aqui usando por una lado lista y por otro df_artista, obtengo una lista de diccionarios con la combinacion de el idArtista que esta en el clasificador de Artistas y el idAudiovisual que es el id que del aoudiovisual en la tabla audiovisual. Esa es la base para crear una tabla puente entre el clasificador de Artistas y la tabla de audiovisuales y que se va a llamar 'artista_audio'. Los errores son elementos en la columna cast que hay que modficar mas adelante.

In [771]:
lista_artista = []
for artista,i in lista:
  try:
    lista_artista.append({'idArtista':df_artista[df_artista['nombre_artista']==artista].loc[:'id'].values[0][0],
                         'idAudiovisual':i})
  except:
    print('Un error en ',artista)                       

Un error en  Marcus " Big Spence"
Un error en  Kiara "The Voice"
Un error en  
Un error en  
Un error en  Angela Bassett as "Miss Helen"
Un error en  Mauricio Barrientos "Diablito"
Un error en  
Un error en  1
Un error en  2
Un error en  3
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  A
Un error en  Carlos Gallardo
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  1
Un error en  Danny  Mann
Un error en  9m88
Un error en  2Mex
Un error en  Carlos Librado "Nene"
Un error en  Carlos Librado "Nene"
Un error en  2 Chainz
Un error en  4Minute


In [ ]:
lista_artista

Creo el DataFrame df_artista_audio

In [773]:
df_artista_audio = pd.DataFrame(lista_artista,index=range(1,len(lista_artista)+1),columns=['idArtista','idAudiovisual'])

Creo una lista de diccionarios que contiene la relacion de cada audiovisual con con cada uno de los generos a que pertenece contenidos en la columna listed_in. Esta relacion es de 1 audiovisual - muchos generos. Para ello me auxilio del clasificador df_genero

In [706]:
lista = []
for i in range(1,df.shape[0]+1):
  lista_genero = df.at[i,'listed_in'].split(',')
  nueva_lista = []                               # creo una lista vacia
  for genero in lista_genero:                      # recorro cada elemento de la lista donde estan los generos
    genero = genero.strip()
    lista.append((genero,i))

In [733]:
df_genero[df_genero['nombre_genero']=='International'].loc[:'id'].values[0][0]

3

In [734]:
lista_genero = []
for genero,i in lista:
  lista_genero.append({'idGenero':df_genero[df_genero['nombre_genero']==genero].loc[:'id'].values[0][0],
                       'idAudiovisual':i})

In [ ]:
lista_genero

Creo el DataFrame df_genero_audio a partis de la lista que tiene los id del cladificador de Generos y el id del audiovisial en la tabla audiovisual

In [748]:
df_genero_audio = pd.DataFrame(lista_genero,index=range(1,len(lista_genero)+1),columns=['idGenero','idAudiovisual'])

Despues de haber creado los clasificadores 'generos', 'artistas' y las tablas 'genero_audio', 'artista_audio' que los relacionan con la tabla 'audiovisual' ya puedo eliminar las columnas cast y listed_in de la tabla audiovisual (df)

In [752]:
df = df[['title','type','release_year','duration','plataforma']]

In [759]:
df_genero = df_genero.set_index('id')

Envio todo lo creado a la BD SQlite 'audiovisuales', alli voy a crear las vistas que me respondan o al menos me ayuden a solucinar las cuatro preguntas que me hacen.
En total me salen 6 tablas
audiovisual   - Tiene la informacion principal de las cuatro plataformas juntas
plataforma    - clasificador de plataformas
genero        - clasificador de generos (lo que estaba en listed_in)
artista       - clasificador de artistas (lo que estaba en cast)
artista_audio - Para relacionar la tabla audiovisual con el clasificador artista
genero_audio  - Para relacionar la tabla audiovisual con el clasificador genero

In [775]:
conn = sqlite3.connect('audiovisuales.db')
df.to_sql('audiovisual', conn, if_exists='replace', index=True)
df_plataforma.to_sql('plataforma', conn, if_exists='replace', index=True)
df_genero.to_sql('genero', conn, if_exists='replace', index=True)
df_artista.to_sql('artista', conn, if_exists='replace', index=False)
df_artista_audio.to_sql('artista_audio', conn, if_exists='replace', index=False)
df_genero_audio.to_sql('genero_audio', conn, if_exists='replace', index=False)
conn.close()

En SQLiteStudio ejecute una consulta para darle valor al campo duration que yo lo habia puesto en 0. Chequeando vi que todos eran del type = 'Movie' asi que calcule el promedio de la columna duration de todas las Movie para ponerle.